<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [ ]:
import nibabel as nib
import numpy as np
import os
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle
"""
package to work out RMSE and SSIM
"""
from utils import calc_RMSE, calc_ssim, load_nii_image, filter_mask, calc_percent

<h2>The class for parsing command line arguments</h2>

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

<h2>Data Preprocessing</h2>

In [ ]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
# s03 data repo
s03_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/'
s03_NDI_path = os.path.join(s03_path, 'corrected_ndi.nii')
s03_ODI_path = os.path.join(s03_path, 'corrected_odi.nii')
s03_FWF_path = os.path.join(s03_path, 'corrected_fwf.nii')

In [ ]:
s03_mask_path = os.path.join(s03_path, 'filtered_mask.nii')

In [ ]:
s03_mask = load_nii_image(s03_mask_path)

In [ ]:
# load the truth data for subject 3
s03_NDI_img = nib.load(s03_NDI_path)
s03_ODI_img = nib.load(s03_ODI_path)
s03_FWF_img = nib.load(s03_FWF_path)
s03_NDI_affine = s03_NDI_img.affine
s03_ODI_affine = s03_ODI_img.affine
s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = s03_NDI_img.get_fdata()
s03_ODI_img_data = s03_ODI_img.get_fdata()
s03_FWF_img_data = s03_FWF_img.get_fdata()

------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [ ]:
# a list contains the N for each tested retained volume
# N = [60, 40]
# random_dict = {60:[], 40:[]}
# random_dict

In [ ]:
# Generated the random scheme for each tested volume
# for n in N:
#     # for each retained volume, we create 100 subsamples
#     for i in range(100):
#         ones = np.ones(n) # 1 = retained
#         zeros = np.zeros(96-n)

#         scheme = np.random.choice(np.concatenate([ones,zeros]), 96, replace=False)
#         scheme = ' '.join(map(str, scheme))
#         random_dict[n].append(scheme)

In [ ]:
# save the rejection scheme
# random_60_file = 'random_60.pickle'
# random_40_file = 'random_40.pickle'

# with open(random_60_file, 'wb') as handle:
#     pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_40_file, 'wb') as handle:
#     pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# If the random rejection is already set, we can load it
random_60 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_60.pickle', "rb")) as openfile:
    while True:
        try:
            random_60.append(pickle.load(openfile))
        except EOFError:
            break
random_60 = np.array(random_60[0])

random_40 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_40.pickle', "rb")) as openfile:
    while True:
        try:
            random_40.append(pickle.load(openfile))
        except EOFError:
            break
random_40 = np.array(random_40[0])

random_30 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_30.pickle', "rb")) as openfile:
    while True:
        try:
            random_30.append(pickle.load(openfile))
        except EOFError:
            break
random_30 = np.array(random_30[0])

In [ ]:
amico_rmse_dict = {60:[], 40:[], 30:[]}
amico_ssim_dict = {60:[], 40:[], 30:[]}
amico_percent_dict = {60:[], 40:[], 30:[]}

----------------------------
<h2>Model fit AMICO</h2>

In [ ]:
# re-fit to ensure the header is preserved
def writefile(path, file, combine, savename):
    with open(os.path.join(path, savename), 'w') as fout:
        read_path = os.path.join(path, file)
        read_file = open(read_path, 'r')
        lines = read_file.readlines()
        for line in lines:
            temp = line.split()
            temp = [e for e, b in zip(temp, combine) if b == 1]
            fout.write(' '.join(e for e in temp))
            fout.write('\n')
    fout.close()

def writediffusion(path, file, combine, savename):
    # the header now is presevred
    img = nib.load(os.path.join(path, file))
    affine = img.affine
    data = img.get_fdata()
    data = data[..., combine==1]
    img = nib.Nifti1Image(data, affine)
    nib.save(img, os.path.join(path, savename))

def run_amico(study_path, subject_path, diffusion, bvals, bvecs, suffix):
    # run amico
    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(os.path.join(study_path, subject_path, bvals), os.path.join(study_path, subject_path, bvecs))
    ae = amico.Evaluation(study_path, subject_path)
    # load the data
    ae.load_data(dwi_filename = diffusion, scheme_filename = bvals+'.scheme', mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels(regenerate=True)
    ae.load_kernels()
    # model fit.
    ae.fit()
    ae.save_results(path_suffix=suffix)

In [ ]:
study_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/'
subject_path = 's03_still_reg'

path = os.path.join(study_path, subject_path)

In [ ]:
save_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/AMICO/'

----------------------
<h2>N=60</h2>

In [ ]:
for i in range(len(random_60)//2):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '60_volume'
    bvals = '60_bvals'
    bvecs = '60_bvecs'
    diffusion = '60_diffusion'

    writefile(path, 'bvals', combine, savename=bvals)
    writefile(path, 'bvecs', combine, savename=bvecs)
    writediffusion(path, 'diffusion.nii', combine, savename=diffusion)

    run_amico(study_path, subject_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)
    amico_percent_dict[60].append(PERCENT)

In [ ]:
with open(save_path+'ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_60)//2, len(random_60)):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '60_volume'
    bvals = '60_bvals'
    bvecs = '60_bvecs'
    diffusion = '60_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)
    amico_percent_dict[60].append(PERCENT)

In [ ]:
with open(save_path+'ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

------
<h2>N=40</h2>

In [ ]:
for i in range(len(random_40)//2):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '40_volume'
    bvals = '40_bvals'
    bvecs = '40_bvecs'
    diffusion = '40_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[40].append(RMSE)
    amico_ssim_dict[40].append(SSIM)
    amico_percent_dict[40].append(PERCENT)


In [ ]:
with open(save_path+'ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_40)//2, len(random_40)):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '40_volume'
    bvals = '40_bvals'
    bvecs = '40_bvecs'
    diffusion = '40_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[40].append(RMSE)
    amico_ssim_dict[40].append(SSIM)
    amico_percent_dict[40].append(PERCENT)

In [ ]:
with open(save_path+'ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h1>N=30</h1>

In [ ]:
for i in range(len(random_30)//2):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '30_volume'
    bvals = '30_bvals'
    bvecs = '30_bvecs'
    diffusion = '30_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[30].append(RMSE)
    amico_ssim_dict[30].append(SSIM)
    amico_percent_dict[30].append(PERCENT)

In [ ]:
with open(save_path+'ami_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_30)//2, len(random_30)):

    # read the movefile
    movefile = random_30[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '30_volume'
    bvals = '30_bvals'
    bvecs = '30_bvecs'
    diffusion = '30_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = nib.load(icvf_path).get_fdata()
    od = nib.load(od_path).get_fdata()
    isovf = nib.load(isovf_path).get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    PERCENT = []
    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    PERCENT.append(ndi_percent)
    PERCENT.append(odi_percent)
    PERCENT.append(fwf_percent)

    amico_rmse_dict[30].append(RMSE)
    amico_ssim_dict[30].append(SSIM)
    amico_percent_dict[30].append(PERCENT)

In [ ]:
with open(save_path+'ami_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_percent_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_percent_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)